In [ ]:
from os import getenv
from aleph_alpha_client import Client
from intelligence_layer.qa import Qa, QaInput, QaOutput
from pprint import pprint

client = Client(getenv("AA_API_TOKEN"))
qa = Qa(client, "debug")
input = QaInput(text="Paul Nicolas lost his mother at the age of 3, and then his father in 1914.[3] He was raised by his mother-in-law together with his brother Henri. He began his football career with Saint-Mandé Club in 1916. Initially, he played as a defender, but he quickly realized that his destiny laid at the forefront since he scored many goals.[3] In addition to his goal-scoring instinct, Nicolas also stood out for his strong character on the pitch, and these two qualities combined eventually drew the attention of Mr. Fort, the then president of the Gallia Club, who signed him as a centre-forward in 1916.",
        question="What is the name of Paul Nicolas' brother?")
output = qa.run(input)

pprint(output.model_dump())